In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from copy import copy

In [5]:
def segmentation_video(vid_path, thresh, median_or_mean='median', filter_size=3):
    cap = cv2.VideoCapture(vid_path)

    ret, frame = cap.read()
    YY = frame.shape[0]
    XX = frame.shape[1]
    N = 60
    iN = 0
    buff = np.zeros((YY, XX, N), np.uint8)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    i = 1  # frame_counter
    
    with open("temporalROI.txt", "r") as f:  # open file
        line = f.readline()  # read line

    roi_start, roi_end = line.split()  # split line
    roi_start = int(roi_start)  # conversion to int
    roi_end = int(roi_end)  # conversion to int

    while True:
        ret, frame = cap.read()  # Odczyt kolejnej klatki
        if not ret:  # Jeśli nie ma więcej klatek, przerwij pętlę
            break
        
        IG = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        buff[:, :, iN] = IG
        iN += 1
        if iN >= N:
            iN = 0
            
        Im = np.mean(buff[:, :, :np.min((N, i))], axis=2).astype(np.uint8) if median_or_mean == 'mean' else np.median(buff[:, :, :np.min((N, i))], axis=2).astype(np.uint8)

        # cv2.imshow("mean", Imean)
        # cv2.imshow("median", Imedian)
        
        Idiff = cv2.absdiff(IG, Im)
        Ibin = (255 * (Idiff > thresh)).astype(np.uint8)
        Ibin = cv2.medianBlur(Ibin, filter_size)
        Imorph = cv2.morphologyEx(Ibin, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        Imorph = cv2.morphologyEx(Imorph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))
        
        # cv2.imshow("morphology", Imorph)
        
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(Imorph)
        # retval -- total number of unique labels
        # labels -- destination labeled image
        # stats -- statistics output for each label , including the background label .
        # centroids -- centroid output for each label , including the background label .

        cv2.imshow("Labels", np.uint8(labels / retval * 255))
        cv2.imshow("Bg", Im)

        if stats.shape[0] > 1:  # are there any objects
            tab = stats[1:, 4]  # 4 columns without first element
            pi = np.argmax(tab)  # finding the index of the largest item
            pi = pi + 1  # increment because we want the index in stats , not in tab
            # drawing a bbox
            cv2.rectangle(
                frame,
                (stats[pi, 0], stats[pi, 1]),
                (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                (255, 0, 0),
                2,
            )
            # print information about the field and the number of the largest element
            cv2.putText(
                frame,
                "%f" % stats[pi, 4],
                (stats[pi, 0], stats[pi, 1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
            )
            cv2.putText(
                frame,
                "%d" % pi,
                (int(centroids[pi, 0]), int(centroids[pi, 1])),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
            )

            cv2.imshow("indexation", frame)
            
            if i > roi_start and i < roi_end:
                gt_mask = cv2.imread("groundtruth/gt%06d.png" % i, 0)   # 0 to szary obraz
                min_row = np.min((Imorph.shape[0], gt_mask.shape[0]))
                min_col = np.min((Imorph.shape[1], gt_mask.shape[1]))
                tp_m = np.logical_and((Imorph[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 255)) * 1  # logical product of the matrix elements
                tn_m = np.logical_and((Imorph[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 0)) * 1
                fp_m = np.logical_and((Imorph[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 0)) * 1
                fn_m = np.logical_and((Imorph[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 255)) * 1

                tp = np.sum(tp_m) + tp  # sum of the elements in the matrix and update sum
                tn = np.sum(tn_m) + tn
                fp = np.sum(fp_m) + fp
                fn = np.sum(fn_m) + fn
            
        # Zakończenie po naciśnięciu klawisza 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit")
            break
        
        i += 1

    # Zwolnienie zasobów
    cap.release()
    cv2.destroyAllWindows()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")
    



In [6]:
path = "pedestrians_input.mp4"
segmentation_video(path, 15, 'median')

precision: 0.8121600216683286
recall: 0.9382748052728941
f1: 0.8706742741320792


In [ ]:
path = "pedestrians_input.mp4"
segmentation_video(path, 15, 'mean')

precision: 0.2253674290586087
recall: 0.9374348103518093
f1: 0.3633761029324328


In [ ]:
def segmentation_images(dir, thresh, median_or_mean='median', filter_size=3):
    frame = cv2.imread("%sinput/in000001.jpg" % (dir))
    YY = frame.shape[0]
    XX = frame.shape[1]
    N = 60
    iN = 0
    buff = np.zeros((YY, XX, N), np.uint8)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    with open("temporalROI.txt", "r") as f:  # open file
        line = f.readline()  # read line

    roi_start, roi_end = line.split()  # split line
    roi_start = int(roi_start)  # conversion to int
    roi_end = int(roi_end)  # conversion to int

    for i in range(roi_start + 1, roi_end):
        frame = cv2.imread("%sinput/in%06d.jpg" % (dir, i))
        
        IG = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        buff[:, :, iN] = IG
        iN += 1
        if iN >= N:
            iN = 0
            
        Im = np.mean(buff[:, :, :np.min((N, i))], axis=2).astype(np.uint8) if median_or_mean == 'mean' else np.median(buff[:, :, :np.min((N, i))], axis=2).astype(np.uint8)

        # cv2.imshow("mean", Imean)
        # cv2.imshow("median", Imedian)
        
        Idiff = cv2.absdiff(IG, Im)
        Ibin = (255 * (Idiff > thresh)).astype(np.uint8)
        Ibin = cv2.medianBlur(Ibin, filter_size)
        Imorph = cv2.morphologyEx(Ibin, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        Imorph = cv2.morphologyEx(Imorph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))
        
        # cv2.imshow("morphology", Imorph)
        
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(Imorph)
        # retval -- total number of unique labels
        # labels -- destination labeled image
        # stats -- statistics output for each label , including the background label .
        # centroids -- centroid output for each label , including the background label .

        cv2.imshow("Labels", np.uint8(labels / retval * 255))

        if stats.shape[0] > 1:  # are there any objects
            tab = stats[1:, 4]  # 4 columns without first element
            pi = np.argmax(tab)  # finding the index of the largest item
            pi = pi + 1  # increment because we want the index in stats , not in tab
            # drawing a bbox
            cv2.rectangle(
                frame,
                (stats[pi, 0], stats[pi, 1]),
                (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                (255, 0, 0),
                2,
            )
            # print information about the field and the number of the largest element
            cv2.putText(
                frame,
                "%f" % stats[pi, 4],
                (stats[pi, 0], stats[pi, 1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
            )
            cv2.putText(
                frame,
                "%d" % pi,
                (int(centroids[pi, 0]), int(centroids[pi, 1])),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
            )

            cv2.imshow("indexation", frame)
            
            if i > roi_start and i < roi_end:
                gt_mask = cv2.imread("%sgroundtruth/gt%06d.png" % (dir, i), 0)   # 0 to szary obraz
                min_row = np.min((Imorph.shape[0], gt_mask.shape[0]))
                min_col = np.min((Imorph.shape[1], gt_mask.shape[1]))
                tp_m = np.logical_and((Imorph[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 255)) * 1  # logical product of the matrix elements
                tn_m = np.logical_and((Imorph[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 0)) * 1
                fp_m = np.logical_and((Imorph[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 0)) * 1
                fn_m = np.logical_and((Imorph[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 255)) * 1

                tp = np.sum(tp_m) + tp  # sum of the elements in the matrix and update sum
                tn = np.sum(tn_m) + tn
                fp = np.sum(fp_m) + fp
                fn = np.sum(fn_m) + fn
            
        # Zakończenie po naciśnięciu klawisza 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit")
            break

    cv2.destroyAllWindows()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")

In [17]:
segmentation_images("office/", 18, 'median')

Exit
precision: nan
recall: nan
f1: nan


C:\Users\roman\AppData\Local\Temp\ipykernel_1964\518622368.py:103: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\roman\AppData\Local\Temp\ipykernel_1964\518622368.py:104: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)


In [39]:
segmentation_images("office/", 18, 'mean')

precision: 0.5694827935041983
recall: 0.5402036279631947
f1: 0.5544569441640632


In [9]:
def segmentation_video_approx(vid_path, thresh, median_or_mean='median', filter_size=3, alfa = 0.01):
    cap = cv2.VideoCapture(vid_path)

    ret, Im = cap.read()
    Im = cv2.cvtColor(Im, cv2.COLOR_BGR2GRAY).astype(np.float64)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    i = 1  # frame_counter
    
    with open("temporalROI.txt", "r") as f:  # open file
        line = f.readline()  # read line

    roi_start, roi_end = line.split()  # split line
    roi_start = int(roi_start)  # conversion to int
    roi_end = int(roi_end)  # conversion to int

    while True:
        ret, frame = cap.read()  # Odczyt kolejnej klatki
        if not ret:  # Jeśli nie ma więcej klatek, przerwij pętlę
            break
        
        IG = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float64)
        
        # średnia
        if median_or_mean == 'mean':
            Im = alfa * IG + (1 - alfa) * Im
        # mediana 
        else:
            Im[Im < IG] += 1
            Im[Im > IG] -= 1
            
        cv2.imshow("Bg", Im.astype(np.uint8))
        
        Idiff = cv2.absdiff(IG.astype(np.uint8), Im.astype(np.uint8))
        Ibin = (255 * (Idiff > thresh)).astype(np.uint8)
        Ibin = cv2.medianBlur(Ibin, filter_size)
        Imorph = cv2.morphologyEx(Ibin, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        Imorph = cv2.morphologyEx(Imorph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))
        
        # cv2.imshow("morphology", Imorph)
        
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(Imorph)
        # retval -- total number of unique labels
        # labels -- destination labeled image
        # stats -- statistics output for each label , including the background label .
        # centroids -- centroid output for each label , including the background label .

        cv2.imshow("Labels", np.uint8(labels / retval * 255))

        if stats.shape[0] > 1:  # are there any objects
            tab = stats[1:, 4]  # 4 columns without first element
            pi = np.argmax(tab)  # finding the index of the largest item
            pi = pi + 1  # increment because we want the index in stats , not in tab
            # drawing a bbox
            cv2.rectangle(
                frame,
                (stats[pi, 0], stats[pi, 1]),
                (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                (255, 0, 0),
                2,
            )
            # print information about the field and the number of the largest element
            cv2.putText(
                frame,
                "%f" % stats[pi, 4],
                (stats[pi, 0], stats[pi, 1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
            )
            cv2.putText(
                frame,
                "%d" % pi,
                (int(centroids[pi, 0]), int(centroids[pi, 1])),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
            )

            cv2.imshow("indexation", frame)
            
            if i > roi_start and i < roi_end:
                gt_mask = cv2.imread("groundtruth/gt%06d.png" % i, 0)   # 0 to szary obraz
                min_row = np.min((Imorph.shape[0], gt_mask.shape[0]))
                min_col = np.min((Imorph.shape[1], gt_mask.shape[1]))
                tp_m = np.logical_and((Imorph[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 255)) * 1  # logical product of the matrix elements
                tn_m = np.logical_and((Imorph[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 0)) * 1
                fp_m = np.logical_and((Imorph[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 0)) * 1
                fn_m = np.logical_and((Imorph[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 255)) * 1

                tp = np.sum(tp_m) + tp  # sum of the elements in the matrix and update sum
                tn = np.sum(tn_m) + tn
                fp = np.sum(fp_m) + fp
                fn = np.sum(fn_m) + fn
            
        # Zakończenie po naciśnięciu klawisza 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit")
            break
        
        i += 1

    # Zwolnienie zasobów
    cap.release()
    cv2.destroyAllWindows()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")
    

In [11]:
segmentation_video_approx("pedestrians_input.mp4", 18, median_or_mean='median', alfa = 0.01)

precision: 0.8669879708745404
recall: 0.9075355440786457
f1: 0.8867985047558237


In [13]:
segmentation_video_approx("pedestrians_input.mp4", 18, median_or_mean='mean', alfa = 0.01)

precision: 0.7979654452463443
recall: 0.9139435288920521
f1: 0.8520258564908987


In [14]:
def segmentation_video_approx_conservative(vid_path, thresh, median_or_mean='median', filter_size=3, alfa = 0.01):
    cap = cv2.VideoCapture(vid_path)

    ret, Im = cap.read()
    Im = cv2.cvtColor(Im, cv2.COLOR_BGR2GRAY).astype(np.float64)
    Imorph = np.ones(Im.shape, dtype=np.uint8)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    i = 1  # frame_counter
    
    with open("temporalROI.txt", "r") as f:  # open file
        line = f.readline()  # read line

    roi_start, roi_end = line.split()  # split line
    roi_start = int(roi_start)  # conversion to int
    roi_end = int(roi_end)  # conversion to int

    while True:
        ret, frame = cap.read()  # Odczyt kolejnej klatki
        if not ret:  # Jeśli nie ma więcej klatek, przerwij pętlę
            break
        
        IG = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float64)
        
        # średnia
        if median_or_mean == 'mean':
            # Im_prev = copy(Im)
            Im[np.logical_not(Imorph)] = alfa * IG + (1 - alfa) * Im
        # mediana 
        else:
            Im[np.logical_and(Im < IG, np.logical_not(Imorph))] += 1
            Im[np.logical_and(Im > IG, np.logical_not(Imorph))] -= 1
            
        cv2.imshow("Im", Im.astype(np.uint8))
        
        Idiff = cv2.absdiff(IG.astype(np.uint8), Im.astype(np.uint8))
        Ibin = (255 * (Idiff > thresh)).astype(np.uint8)
        Ibin = cv2.medianBlur(Ibin, filter_size)
        Imorph = cv2.morphologyEx(Ibin, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        Imorph = cv2.morphologyEx(Imorph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))
        
        # cv2.imshow("morphology", Imorph)
        
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(Imorph)
        # retval -- total number of unique labels
        # labels -- destination labeled image
        # stats -- statistics output for each label , including the background label .
        # centroids -- centroid output for each label , including the background label .

        cv2.imshow("Labels", np.uint8(labels / retval * 255))

        if stats.shape[0] > 1:  # are there any objects
            tab = stats[1:, 4]  # 4 columns without first element
            pi = np.argmax(tab)  # finding the index of the largest item
            pi = pi + 1  # increment because we want the index in stats , not in tab
            # drawing a bbox
            cv2.rectangle(
                frame,
                (stats[pi, 0], stats[pi, 1]),
                (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                (255, 0, 0),
                2,
            )
            # print information about the field and the number of the largest element
            cv2.putText(
                frame,
                "%f" % stats[pi, 4],
                (stats[pi, 0], stats[pi, 1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
            )
            cv2.putText(
                frame,
                "%d" % pi,
                (int(centroids[pi, 0]), int(centroids[pi, 1])),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
            )

            cv2.imshow("indexation", frame)
            
            if i > roi_start and i < roi_end:
                gt_mask = cv2.imread("groundtruth/gt%06d.png" % i, 0)   # 0 to szary obraz
                min_row = np.min((Im.shape[0], gt_mask.shape[0]))
                min_col = np.min((Im.shape[1], gt_mask.shape[1]))
                tp_m = np.logical_and((Im[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 255)) * 1  # logical product of the matrix elements
                tn_m = np.logical_and((Im[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 0)) * 1
                fp_m = np.logical_and((Im[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 0)) * 1
                fn_m = np.logical_and((Im[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 255)) * 1

                tp = np.sum(tp_m) + tp  # sum of the elements in the matrix and update sum
                tn = np.sum(tn_m) + tn
                fp = np.sum(fp_m) + fp
                fn = np.sum(fn_m) + fn
            
        # Zakończenie po naciśnięciu klawisza 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit")
            break
        
        i += 1

    # Zwolnienie zasobów
    cap.release()
    cv2.destroyAllWindows()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")

In [ ]:
segmentation_video_approx_conservative("pedestrians_input.mp4", 18, median_or_mean='median', alfa = 0.01)

TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions

In [19]:
def segmentation_cv2_images(type, dir, history=500, varThreshold=16, detectShadows=True):
    Im = cv2.imread("%sinput/in000001.jpg" % (dir))
    if type == 'MOG':
        OBJ = cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=varThreshold, detectShadows=detectShadows)
    else:
        OBJ = cv2.createBackgroundSubtractorKNN(history=history, dist2Threshold=varThreshold, detectShadows=detectShadows)
    Im = cv2.cvtColor(Im, cv2.COLOR_BGR2GRAY)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    with open("temporalROI.txt", "r") as f:  # open file
        line = f.readline()  # read line

    roi_start, roi_end = line.split()  # split line
    roi_start = int(roi_start)  # conversion to int
    roi_end = int(roi_end)  # conversion to int

    for i in range(roi_start + 1, roi_end):
        frame = cv2.imread("%sinput/in%06d.jpg" % (dir, i))
        IG = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        Imask = OBJ.apply(image=IG, fgmask=Im)
        
        Im = cv2.morphologyEx(Imask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
        Im = cv2.morphologyEx(Im, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(Im)
        # retval -- total number of unique labels
        # labels -- destination labeled image
        # stats -- statistics output for each label , including the background label .
        # centroids -- centroid output for each label , including the background label .

        cv2.imshow("Labels", np.uint8(labels / retval * 255))

        if stats.shape[0] > 1:  # are there any objects
            tab = stats[1:, 4]  # 4 columns without first element
            pi = np.argmax(tab)  # finding the index of the largest item
            pi = pi + 1  # increment because we want the index in stats , not in tab
            # drawing a bbox
            cv2.rectangle(
                frame,
                (stats[pi, 0], stats[pi, 1]),
                (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                (255, 0, 0),
                2,
            )
            # print information about the field and the number of the largest element
            cv2.putText(
                frame,
                "%f" % stats[pi, 4],
                (stats[pi, 0], stats[pi, 1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
            )
            cv2.putText(
                frame,
                "%d" % pi,
                (int(centroids[pi, 0]), int(centroids[pi, 1])),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
            )

            cv2.imshow("indexation", frame)
            
            if i > roi_start and i < roi_end:
                gt_mask = cv2.imread("%sgroundtruth/gt%06d.png" % (dir, i), 0)   # 0 to szary obraz
                min_row = np.min((Im.shape[0], gt_mask.shape[0]))
                min_col = np.min((Im.shape[1], gt_mask.shape[1]))
                tp_m = np.logical_and((Im[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 255)) * 1  # logical product of the matrix elements
                tn_m = np.logical_and((Im[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 0)) * 1
                fp_m = np.logical_and((Im[:min_row, :min_col] == 255), (gt_mask[:min_row, :min_col] == 0)) * 1
                fn_m = np.logical_and((Im[:min_row, :min_col] == 0), (gt_mask[:min_row, :min_col] == 255)) * 1

                tp = np.sum(tp_m) + tp  # sum of the elements in the matrix and update sum
                tn = np.sum(tn_m) + tn
                fp = np.sum(fp_m) + fp
                fn = np.sum(fn_m) + fn
            
        # Zakończenie po naciśnięciu klawisza 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit")
            break

    cv2.destroyAllWindows()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")


In [20]:
# wartości domyślne
segmentation_cv2_images('MOG', 'pedestrian/')

precision: 0.9848729462649807
recall: 0.813914883582386
f1: 0.8912699275608529


In [21]:
segmentation_cv2_images('MOG', 'pedestrian/', history=500, varThreshold=18, detectShadows=False)

Exit
precision: 0.6648507155290397
recall: 0.8289430958960392
f1: 0.737884179495385


In [22]:
# wartości domyślne
segmentation_cv2_images('KNN', 'pedestrian/')

Exit
precision: 0.4232590194883989
recall: 0.9213811099932097
f1: 0.5800553719026761


In [69]:
segmentation_cv2_images('KNN', 'pedestrian/', history=500, varThreshold=20, detectShadows=False)

precision: 0.5471731379218726
recall: 0.9946114103943882
f1: 0.7059671820329618
